In [15]:
!pip install wfdb -q
!pip install scaleogram -q

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow_p36/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow_p36/bin/python -m pip install --upgrade pip' command.


In [1]:
import wfdb
from wfdb import processing
import os
import matplotlib.pyplot as plt
import wfdb_custom
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
from scipy import io
import scaleogram as scg

In [2]:
### Set Dataset Path
dataset_paths = [ "/home/ec2-user/SageMaker/physionet.org/files/challenge-2016/1.0.0/training-a/" , "/home/ec2-user/SageMaker/datasets/Physionet2016_a/DenoisedEMD2016/","/home/ec2-user/SageMaker/datasets/Physionet2016_a/DenoisedEMD_ECG2016/"]
files        = [ os.listdir(dataset_paths[0]), os.listdir(dataset_paths[1]), os.listdir(dataset_paths[2])]

In [3]:
#extract .wav file names only (folder has .hea , .dat as well)
wav_files  = sorted([file for file in files[0] if '.wav' in file])
pcgmat_files  = sorted([file for file in files[1] if '.mat' in file])
ecgmat_files  = sorted([file for file in files[2] if '.mat' in file])

In [4]:
def pcgmat2np(x):
    data = scipy.io.loadmat(dataset_paths[1] + pcgmat_files[x])  # eg: 'D_A00001.mat'
    wave = data['y'] # numpy array
    wave = wave.reshape(wave.shape[0])
    return wave

In [5]:
def ecgmat2np(x):
    data = scipy.io.loadmat(dataset_paths[2] + ecgmat_files[x])  # eg: 'D_A00001.mat'
    wave = data['y'] # numpy array
    wave = wave.reshape(wave.shape[0])
    return wave

In [6]:
def plot_wavelet(a,ecg=False,save_path=None):
    
    
    scales = 0
    if(ecg):
        scales = np.arange(20, 500)
        yscaletype = 'log'
        scg.set_default_wavelet('cmor1.5-1.0')
    else:
        scales = np.arange(7, 130)
        yscaletype = 'log'
        scg.set_default_wavelet('morl')

    fs = 2000
    xcg = a
    t = np.array(range(len(xcg)-1))/fs
    cwt = scg.CWT(t, xcg,scales)
    fig = scg.cws(cwt, figsize=(9,5),yaxis='frequency',yscale = yscaletype,cscale='linear',xlabel="Time (s)",coi=False)
    
    if(save_path!=None):
        plt.savefig(save_path)
    #plt.show()
    plt.clf()
    

In [7]:
export_paths = ["/home/ec2-user/SageMaker/datasets/Physionet2016_a/new_split_basedon_rawpcg/raw/",
                "/home/ec2-user/SageMaker/datasets/Physionet2016_a/new_split_basedon_rawpcg/denoised/"]

In [8]:
def metadata(x,s=100,e=14000):
    '''
        Accepts: a file id, x
        Returns: Prints parameters about a specific record
                 Plots the locally detected peaks
    
    '''
    
    rec = wav_files[x][:-4]
  
    #rec = pcgmat_files[x][2:7]
    
    record = wfdb.rdrecord(dataset_paths[0] +rec) 

    #print(record.sig_name)
    #print(record.comments)
    #print(record.record_name)
    #print(record.fs)
    #print(record.p_signal.shape)
    if(record.p_signal.shape[1]<2):
        print ("chuto")
        return 
    
    a = record.p_signal[:,0]
    a = a[np.logical_not(np.isnan(a))]
    a = a - np.mean(a);
    
    b = record.p_signal[:,1]
    b = b[np.logical_not(np.isnan(b))]
    b = b - np.mean(b);
   

    pcg_p = wfdb.processing.find_local_peaks(a,1000) 
    #ecg_p = wfdb.processing.find_local_peaks(record.p_signal[:,1],1000) 
    
    #peaks = [pcg_p , ecg_p]
    #save_path = export_paths[0] + "pcg/lop.png"
    #wfdb_custom.plot_wfdb(s,e,peaks,record=record, title='Record ' + rec + ' from PhysioNet Challenge 2016' + ' => ' + record.__dict__['comments'][0],figsize=(12,5), time_units = 'samples',ecg_grids=[1]) 
    
    #plot_wavelet(a[s:e])
    #plot_wavelet(b[s:e],ecg=True)

    
    '''
    ##Denoised
    pcg_wave = pcgmat2np(x)
    ecg_wave = ecgmat2np(x)
    pcg_wave = pcg_wave - np.mean(pcg_wave)
    ecg_wave = ecg_wave - np.mean(ecg_wave)
    
    
    if(len(pcg_wave)==0 or len(ecg_wave)==0):
        print("==== ERROR ==== ", x)
        return
    '''
    
    #pcg_p = wfdb.processing.find_local_peaks(pcg_wave,1000) 
    #ecg_p = wfdb.processing.find_local_peaks(ecg_wave,1000) 
    #peaks = [pcg_p , ecg_p] 
    
    #wave = [pcg_wave , ecg_wave]
    
    
    #print("Denoised Waves")
    #wfdb_custom.custom_plot_signal(s,e,peaks,wave,figsize=(12,5))
    #plot_wavelet(pcg_wave[s:e] )
    #plot_wavelet(ecg_wave[s:e],ecg=True)
    
    
    e_p = -1
    
    
    #Scalogram Generation
    for i in range(0,len(pcg_p)):

        s_p = pcg_p[i]
        if(s_p < e_p):
            continue
        
        e_p   = s_p + 7000
        #print(s_p,e_p)
        
        rec = wav_files[x][:-4]
        if(record.comments[0]=='Abnormal'):
            rec = rec + "_" + str(s_p) + '_A.tiff'
        else:
            rec = rec + "_" + str(s_p) + '_N.tiff'
         
        plot_wavelet(a[s_p:e_p]   ,save_path=export_paths[0] + 'pcg/' + rec )
        plot_wavelet(b[s_p:e_p]   ,ecg=True,save_path=export_paths[0] + 'ecg/' + rec )
        
        #plot_wavelet(pcg_wave[s_p:e_p],save_path=export_paths[1] + 'pcg/' + rec )
        #plot_wavelet(ecg_wave[s_p:e_p],ecg=True,save_path=export_paths[1] + 'ecg/' + rec )
        
   
    
     

In [10]:
print("j")

j


In [ ]:
for i in range(218,409):
    metadata(i)
    print(i)

218
chuto
219


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/scaleogram/cws.py:142: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(figsize=figsize)


220
221
222
223
224
225
226
227
228
229
230
231
chuto
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408


<Figure size 648x360 with 0 Axes>

<Figure size 648x360 with 0 Axes>

...

<Figure size 648x360 with 0 Axes>

<Figure size 648x360 with 0 Axes>

<Figure size 648x360 with 0 Axes>

<Figure size 648x360 with 0 Axes>

<Figure size 648x360 with 0 Axes>

In [ ]:
#PCG => Morlet 

#ECG => Cmor1.5-1.0

'''
    if(ecg):
        scales = np.arange(2, 450)
        yscaletype = 'log'
        scg.set_default_wavelet('cmor1.5-1.0')
    else:
        scales = np.arange(7, 130)
        yscaletype = 'log'
        scg.set_default_wavelet('morl')
'''



In [ ]:
wave = mat2np(26)
plt.plot(wave)

In [14]:
count = 0 
for x in range(0,len(pcgmat_files)):
    pcg_wave = pcgmat2np(x)
    ecg_wave = ecgmat2np(x)
    pcg_wave = pcg_wave - np.mean(pcg_wave)
    ecg_wave = ecg_wave - np.mean(ecg_wave)
    
    if(len(pcg_wave)==0 or len(ecg_wave)==0):
        count = count+1
        print("==== ERROR ==== ", x)
    

==== ERROR ====  12
==== ERROR ====  13
==== ERROR ====  17
==== ERROR ====  26
==== ERROR ====  27
==== ERROR ====  43
==== ERROR ====  53
==== ERROR ====  55
==== ERROR ====  66
==== ERROR ====  68
==== ERROR ====  73
==== ERROR ====  88
==== ERROR ====  111
==== ERROR ====  115
==== ERROR ====  134
==== ERROR ====  156
==== ERROR ====  157
==== ERROR ====  160
==== ERROR ====  176
==== ERROR ====  182
==== ERROR ====  201
==== ERROR ====  224
==== ERROR ====  245
==== ERROR ====  256
==== ERROR ====  268
==== ERROR ====  269
==== ERROR ====  271
==== ERROR ====  274
==== ERROR ====  286
==== ERROR ====  290
==== ERROR ====  298
==== ERROR ====  300
==== ERROR ====  306
==== ERROR ====  309
==== ERROR ====  310
==== ERROR ====  315
==== ERROR ====  328
==== ERROR ====  332
==== ERROR ====  339
==== ERROR ====  342
==== ERROR ====  356
==== ERROR ====  361
==== ERROR ====  362
==== ERROR ====  390
==== ERROR ====  395


In [15]:
count

45

In [ ]:
#!aws s3 cp /home/ec2-user/SageMaker/datasets s3://intelliscope-data/datasets --recursive